# Hessian Matrix

In [5]:
import pandas as pd
import numpy as np
import re
from sympy import *

In [8]:
def derivate(func_str):
    x = Symbol('x')
    return str(diff(eval(func_str), x))

In [9]:
derivate("2*x**3+x**2-4")

'6*x**2 + 2*x'

In [10]:
def is_convex(func_str):
    return bool